In [ ]:
%matplotlib inline
%run ../../setup/nb_setup

# Orbits 3: Orbits in Triaxial Potentials

## Solutions to Exercises

In [ ]:
from astropy.constants import G
import astropy.units as u

from IPython.display import HTML
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

### Exercise: Define a Triaxial Log Potential with Gala

Define a Gala `LogarithmicPotential` object with:
* $v_c=230~\textrm{km}~\textrm{s}^{-1}$
* $r_h=15~\textrm{kpc}$
* $q_1 = 1$
* $q_2 = 0.9$
* $q_3 = 0.8$

In [ ]:
triaxial_log = gp.LogarithmicPotential(
    v_c=230 * u.km / u.s, r_h=15 * u.kpc, q1=1.0, q2=0.9, q3=0.8, units=galactic
)

### Exercise: Long- and short-axis tube orbits

Define a `PhaseSpacePosition` object to represent two initial conditions: 

* At $\boldsymbol{x} = (10, 1, 3)~\textrm{kpc}$, with $v_y = v_c$
* At $\boldsymbol{x} = (3, 10, 1)~\textrm{kpc}$, with $v_z = v_c$

(other velocity components set to 0)

In [ ]:
tube_w0s = gd.PhaseSpacePosition(
    pos=([[10, 1, 3.0], [3, 10, 1.0]] * u.kpc).T,
    vel=([[0, 230, 0.0], [0, 0, 230.0]] * u.km / u.s).T,
)

Integrate these orbits in the `triaxial_log` potential defined above for a total integration time of 100 Gyr with a timestep of 2 Myr:

In [ ]:
tube_orbits = triaxial_log.integrate_orbit(
    tube_w0s, dt=2 * u.Myr, t1=0, t2=100 * u.Gyr, Integrator=gi.DOPRI853Integrator
)

Plot the two orbits in all 2D projections of the 3D positions (x-y, x-z, y-z) on separate figures:

In [ ]:
for i in range(tube_orbits.norbits):
    fig = tube_orbits[:, i].plot()
    fig.suptitle(f"Orbit {i+1}")

Compute and plot the angular momentum components for the two orbits as a function of time:

In [ ]:
tube_orbits_L = tube_orbits.angular_momentum()

In [ ]:
fig, axes = plt.subplots(
    2, 1, figsize=(6, 8), sharex=True, sharey=True, constrained_layout=True
)
for i, ax in enumerate(axes):
    for k, lbl in enumerate(["$L_x$", "$L_y$", "$L_z$"]):
        ax.plot(tube_orbits.t, tube_orbits_L[k, :, i], label=lbl)
    ax.set_title(f"Orbit {i+1}")

axes[0].legend(ncol=3, loc="best")
axes[-1].set_xlabel(f"time [{tube_orbits.t.unit:latex_inline}]")

What differences do you see in the time-series angular momentum components as compared to orbits in an axisymmetric model?

*Answer:* None of the angular momentum components are exactly conserved! However: the angular momentum component corresponding to the circulation axis for each orbit is approximately conserved: It at least has a constant sign and appears to oscillate around a mean value.

### Exercise: Tube orbits around the intermediate axis?

Set up initial conditions to compute a tube orbit around the intermediate axis, starting from the position $\boldsymbol{x} = (10, 0.5, 0)~\textrm{kpc}$.

In [ ]:
y_tube_w0 = gd.PhaseSpacePosition(
    pos=[10, 0.5, 0] * u.kpc, vel=[0, 0, 230.0] * u.km / u.s
)

Integrate this orbit for the same time array as the tube orbits we computed above:

In [ ]:
y_tube_orbit = triaxial_log.integrate_orbit(
    y_tube_w0, t=tube_orbits.t, Integrator=gi.DOPRI853Integrator
)

Plot the orbit in projections:

In [ ]:
_ = y_tube_orbit.plot()

Compute the angular momentum components and plot them:

In [ ]:
y_tube_L = y_tube_orbit.angular_momentum()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4), constrained_layout=True)
for k, lbl in enumerate(["$L_x$", "$L_y$", "$L_z$"]):
    ax.plot(y_tube_orbit.t, y_tube_L[k, :], label=lbl)

ax.legend(ncol=3, loc="best")
ax.set_xlabel(f"time [{tube_orbits.t.unit:latex_inline}]")

What is different about the angular momentum component time series for this orbit?

### Exercise: A grid of orbits with equal energy

In this exercise, we are going to compute a grid of orbits started with the same total energy to map out the orbit structure of a portion of phase-space. How should we choose the initial conditions for our grid of orbits? We need to set the 6 phase-space coordinates for each orbit. Requiring that they have the same energy gives us 1 constraint. To further reduce the dimensionality, we will assume $y=v_x=v_z=0$ (we now have 4 constraints). We will then choose a grid in $x, z$ to set the final two coordinates. At any location in our $x,z$ grid, we will use the energy to determine the value of $v_y$ from:

$$
E = \frac{1}{2}(v_x^2 + v_y^2 + v_z^2) + \Phi(x,y,z)\\
v_y = \sqrt{2\,(E - \Phi(x, 0, z))}
$$

For the energy, we will use $E = 0.195~\textrm{kpc}^2~\textrm{Myr}^{-2}$:

In [ ]:
grid_E = 0.195 * (u.kpc / u.Myr) ** 2

Generate a 1D grid of 41 $x$ values between $(15, 25)~\textrm{kpc}$, and a 1D grid of 41 $z$ values between $(0, 20)~\textrm{kpc}$. Use these 1D grids to construct a 2D grid with all 1,681 pairs of coordinates (*Hint: use `numpy.meshgrid()`*). Store an array of all $x,y,z$ values (all $y$ values are 0) in the variable `grid_pos`:

In [ ]:
_x_grid = np.linspace(15, 25, 41)
_z_grid = np.linspace(0, 20, 41)
grid_shape = (len(_x_grid), len(_z_grid))
# x_grid, z_grid = ...

x_grid, z_grid = np.meshgrid(_x_grid, _z_grid)

grid_pos = np.zeros((3, x_grid.size))
grid_pos[0] = x_grid.ravel()
grid_pos[2] = z_grid.ravel()

Compute the potential energy at all locations in the grid, and use the difference of the grid energy `grid_E` and the potential energy to compute the initial $v_y$:

In [ ]:
grid_Phi = triaxial_log.energy(grid_pos)
vy_grid = np.sqrt(2 * (grid_E - grid_Phi))

(Some of the $v_y$ values may come out as NaN: that is ok, you can ignore those - there are some values of our $x,z$ grid that are outside of the iso-potential-energy surface)

Plot the grid of $x,y$ positions colored by the value of $v_y$ at each location (i.e. the following cell should execute)

In [ ]:
plt.figure(figsize=(7, 6))
plt.pcolormesh(x_grid, z_grid, vy_grid.reshape(grid_shape).to_value(u.km / u.s))
plt.xlabel("$x_0$")
plt.ylabel("$z_0$")

cb = plt.colorbar()
cb.set_label(r"$v_y$")

Set up the full grid of initial conditions as a `PhaseSpacePosition` object named `grid_w0`:

In [ ]:
grid_vel = np.zeros(grid_pos.shape) * u.km / u.s
grid_vel[1] = vy_grid

grid_w0 = gd.PhaseSpacePosition(pos=grid_pos * u.kpc, vel=grid_vel)

Compute the orbits for all of the initial conditions in the grid using the default `LeapfrogIntegrator`, using a timestep of 2 Myr, and integrate for 10 Gyr

In [ ]:
grid_orbits = triaxial_log.integrate_orbit(grid_w0, dt=2 * u.Myr, t1=0, t2=10 * u.Gyr)

Compute the angular momentum components for all orbits, and then compute the peak-to-peak spread in each angular momentum component for each orbit (i.e. compute $\textrm{max}(L_i) - \textrm{min}(L_i)$ for each component $i$ for each orbit)

In [ ]:
grid_orbits_L = grid_orbits.angular_momentum()
ptp_L = np.max(grid_orbits_L, axis=1) - np.min(grid_orbits_L, axis=1)

Make a 3 panel plot (panels corresponding to the 3 angular momentum components) showing a 2D image of the peak-to-peak spread in each component (i.e. the plot commands below should execute)

In [ ]:
fig, axes = plt.subplots(
    1, 3, figsize=(12, 4.5), sharex=True, sharey=True, constrained_layout=True
)
for i, lbl in enumerate(["$L_x$", "$L_y$", "$L_z$"]):
    axes[i].pcolormesh(x_grid, z_grid, ptp_L[i].value.reshape(grid_shape))
    axes[i].set_title(lbl)

for ax in axes:
    ax.set_xlabel("$x_0$")
axes[0].set_ylabel("$z_0$")

* What structure do you see in this diagram? 
* What do you think causes the structure we see in this diagram?
* Can you identify the transition from tube to box orbits?